In [1]:
#!pip install mtcnn

# import the necessary packages
from pyimagesearch.blur_detector import detect_blur_fft
import numpy as np
import argparse
import imutils
import cv2
import mtcnn
import matplotlib.pyplot as plt
from imutils import paths
import argparse
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import PySimpleGUI as sg
import os




################################## BLUR DETECTION #############################

test_image_link = r'1_Test_Images_Blur_Emotions'


# Blur Detection
def BlurDetection(test_image_link):

    # set the blur/clear threshold
    threshold = 10

    int = 0    
    for imagePath in paths.list_images(test_image_link) :
            testimage = plt.imread(imagePath)
            orig = cv2.imread(imagePath)
            orig = imutils.resize(orig, width=500)
            gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
            # apply our blur detector using the FFT
            (mean, blurry) = detect_blur_fft(gray, size=60,thresh=threshold, vis=-1 > 0)   
        
            int = int + 1

            if blurry == True :
                    plt.imsave('1a_Blur_Images/' + str(int) + '.png', testimage , format='png')
            #        os.remove(os.path.join(folder_path, images))
            else :
                    plt.imsave('1b_No_Blur/' + str(int) + '.png', testimage , format='png')


                    
################################## EMOTIONS #############################
                    
# Face Detection and Emotion Classification

def FaceAndEmotion(test_image_link):

    detector = mtcnn.MTCNN()

    # Create the model
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))

    model.load_weights('model_epoch_50_lr0_0001.h5')
    

    # prevents openCL usage and unnecessary logging messages
    cv2.ocl.setUseOpenCL(False)

    # dictionary which assigns each label an emotion (alphabetical order)
    emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

    int = 0    
    for imagePath in paths.list_images(test_image_link) :
            pixels = plt.imread(imagePath)
        
            faces = detector.detect_faces(pixels)
            int = int + 1
            if not faces:
                    plt.imsave('2b_Not_Face/' + str(int) + '.png', pixels , format='png')
                
            else :
                
                    #draw face
                    list = []
                    #plt.imshow(pixels)
                    #ax = plt.gca()
                    for i in faces :
                        x, y, width, height = i['box']
                        list.append(i['box'])
                        rect = plt.Rectangle((x,y), width, height, fill=False, color='red')
                        #ax.add_patch(rect)
                    #plt.axis('off')
                    #plt.savefig(str(int) + '.png', bbox_inches='tight',pad_inches=0)
                    #print(faces)
                
                    #print(list)
                    list = [[0 if x < 0 else x for x in y] for y in list]
                    #print(list)
                    img = cv2.imread(imagePath)
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                    for (x,y,w,h) in list:
                            img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
                            roi_gray = gray[y:y+h, x:x+w]
                            roi_color = img[y:y+h, x:x+w]
                            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
                            prediction = model.predict(cropped_img)
                            maxindex = (np.argmax(prediction))
                            #maxindex = (np.predictions)
                            #cv2.putText(img, emotion_dict[maxindex],  (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                            #cv2.imshow('img',img)
                        
                            if maxindex == 0 : 
                                 plt.imsave('2_Emotions/angry/' + str(int) + '.png',pixels, format='png')                        
                            if maxindex == 1 : 
                                 plt.imsave('2_Emotions/disgusted/' + str(int) + '.png',pixels, format='png')                        
                            if maxindex == 2 : 
                                 plt.imsave('2_Emotions/fearful/' + str(int) + '.png',pixels, format='png')                        
                            if maxindex == 3 : 
                                 plt.imsave('2_Emotions/happy/' + str(int) + '.png',pixels, format='png')
                            if maxindex == 4 : 
                                 plt.imsave('2_Emotions/neutral/' + str(int) + '.png',pixels, format='png')                        
                            if maxindex == 5 : 
                                 plt.imsave('2_Emotions/sad/' + str(int) + '.png',pixels, format='png')                        
                            if maxindex == 6 : 
                                 plt.imsave('2_Emotions/surprised/' + str(int) + '.png',pixels, format='png')
                        
                            #cv2.waitKey(0)
                            #cv2.destroyAllWindows()
                
                    plt.imsave('2a_Face/' + str(int) + '.png',pixels, format='png')   
                    
                    
                    

################################## CLASSIFICATIONS #############################
                    
import tensorflow as tf
from keras.models import load_model

#tensorflow:Model was constructed with shape (None, 160, 160, 3) 
#for input KerasTensor(type_spec=TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='input_2'), 
                                    #name='input_2', description="created by layer 'input_2'"), 
#but it was called on an input with incompatible shape (None, 160).

classification_test_images = r'3_Test_Images_Classification'

# Classification
def Classification(classification_test_images):
    
    model = load_model('MobileNetV2_8classes_model_v3.h5')
    
    # dictionary which assigns each label an emotion (alphabetical order)
    class_dict = {0: "animals", 1: "architecture", 2: "art_and_culture", 3: "food_and_drinks",
                  4: "landscapes", 5: "racing", 6: "sports", 7: "travel_and_adventure"}
    
    int = 0    
    for imagePath in paths.list_images(classification_test_images) :
            pixels = plt.imread(imagePath)
            image = cv2.imread(imagePath)
            #class_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            class_image = np.expand_dims(np.expand_dims(cv2.resize(image, (160,160)), -1), 0)
            predictions = model.predict(class_image) #.flatten()
            
            maxindex = (np.argmax(predictions))
            
            #Retrieve a batch of images from the test set
            
            if maxindex == 0 : 
                  plt.imsave('3a_Classifications/animals/' + str(int)  + '.png', pixels, format='png')                     
            if maxindex == 1 : 
                  plt.imsave('3a_Classifications/architecture/' + str(int) + '.png', pixels, format='png')                        
            if maxindex == 2 : 
                  plt.imsave('3a_Classifications/art_and_culture/' + str(int) + '.png', pixels, format='png')                        
            if maxindex == 3 : 
                  plt.imsave('3a_Classifications/food_and_drinks/' + str(int) + '.png', pixels, format='png')
            if maxindex == 4 : 
                  plt.imsave('3a_Classifications/landscapes/' + str(int) + '.png', pixels, format='png')                        
            if maxindex == 5 : 
                  plt.imsave('3a_Classifications/racing/' + str(int) + '.png', pixels, format='png')                        
            if maxindex == 6 : 
                  plt.imsave('3a_Classifications/sports/' + str(int) + '.png', pixels, format='png')
            if maxindex == 7 : 
                  plt.imsave('3a_Classifications/travel_and_adventure/' + str(int) + '.png', pixels, format='png')

                             #cv2.waitKey(0)
                             #cv2.destroyAllWindows()
                
    #plt.imsave('3a_Classifications/' + '.png', image, format='png') #

#Classification(3a_Classifications)



################################## CLEAR FOLDERS #############################

# Clear All Folders


def ClearFolders():
    fileList = glob.glob(r'2_Emotions\*\*.png')
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)


    folder_path = (r'2b_Not_Face')
    test = os.listdir(folder_path)
    for images in test:
        if images.endswith(".png"):
            os.remove(os.path.join(folder_path, images))


    folder_path = (r'2a_Face')
    test = os.listdir(folder_path)
    for images in test:
        if images.endswith(".png"):
            os.remove(os.path.join(folder_path, images))


    folder_path = (r'1a_Blur_Images')
    test = os.listdir(folder_path)
    for images in test:
        if images.endswith(".png"):
            os.remove(os.path.join(folder_path, images))


    folder_path = (r'1b_No_Blur')
    test = os.listdir(folder_path)
    for images in test:
        if images.endswith(".png"):
            os.remove(os.path.join(folder_path, images))


    fileList1 = glob.glob(r'3a_Classifications\*\*.png')
    for filePath in fileList1:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)


            
################################## SYSTEM INTEGRATION #############################
            
# System Integration
import PySimpleGUI as sg

# Add your new theme colors and settings
my_new_theme = {'BACKGROUND': '#3bbdc2',
                'TEXT': 'white',
                'INPUT': 'white',
                'TEXT_INPUT': '#3D3D3D',
                'SCROLL': '#c7e78b',
                'BUTTON': ('white', '#FE5BAC'), #007489
                'PROGRESS': ('#01826B', '#D0D0D0'),
                'BORDER': 1,
                'SLIDER_DEPTH': 0,
                'PROGRESS_DEPTH': 0}

# Add your dictionary to the PySimpleGUI themes
sg.theme_add_new('MyNewTheme', my_new_theme)

# Switch your theme to use the newly added one. You can add spaces to make it more readable
sg.theme('My New Theme')

def btn(name):  # a PySimpleGUI "User Defined Element" (see docs)
    return sg.Button(name, size=(18, 2), font=("Helvetica", 11), pad=(2, 2))

# Define the window's contents
layout = [[sg.Image(r'ai_logo.png')],
          [sg.Text("")],
          [sg.Text("What's your name?", size=(40, 1), font=("Helvetica", 14))],
          [sg.Input(key='-INPUT-', size=(60, 1))],
          [sg.Text("")],
          [sg.Text("Blur Detection", size=(40, 1), font=("Helvetica", 14))],
          [sg.Input(default_text='Input Local Folder Path', key='-PATH_Blur-', size=(60, 1))],
          [sg.Text("")],
          [sg.Text("Face and Emotion", size=(40, 1), font=("Helvetica", 14))],
          [sg.Input(default_text='Input Local Folder Path', key='-PATH_FaceEmotion-', size=(60, 1))],
          [sg.Text("")],
          [sg.Text("Classify Favourites", size=(40, 1), font=("Helvetica", 14))],
          [sg.Input(default_text='Input Local Folder Path', key='-PATH_Classify-', size=(60, 1))],
          [sg.Text(size=(100,1), key='-OUTPUT-')],
          [sg.Text(size=(100,1), key='-OUTPUTPATHBlur-')],
          [sg.Text(size=(100,1), key='-OUTPUTPATHFaceAndEmotion-')],
          [sg.Text(size=(100,1), key='-OUTPUTPATHClassification-')],
          [sg.Text(size=(100,1), key='-OUTPUTPATHClearFolders-')],
          [btn('Blur'), btn('Face and Emotion'), btn('Classify Favourites'), btn('Clear Albums'), btn('Quit')]]

# Create the window
window = sg.Window('ai@foto', layout)

    
# Display and interact with the Window using an Event Loop
while True:
    event, values = window.read()
    
    if event == 'Blur' :
        BlurDetection(values['-PATH_Blur-'])
        #blur_detection(values['-PATH-'])
        
    if event == 'Face and Emotion' :
        FaceAndEmotion(values['-PATH_FaceEmotion-'])
        #blur_detection(values['-PATH-'])
        
    if event == 'Classify Favourites' :
        Classification(values['-PATH_Classify-'])
        #blur_detection(values['-PATH-'])
        
    if event == 'Clear Albums' :
        ClearFolders()
        #blur_detection(values['-PATH-'])
    
    # See if user wants to quit or window was closed
    if event == sg.WINDOW_CLOSED or event == 'Quit':
        break
        
    # Output a message to the window
    window['-OUTPUT-'].update('Hello ' + values['-INPUT-'] + "! Thanks for using our App")
    window['-OUTPUTPATHBlur-'].update('Your Path is for Blur Detection is :' + values['-PATH_Blur-'])
    window['-OUTPUTPATHFaceAndEmotion-'].update('Your Path is for Face And Emotion is :' + values['-PATH_FaceEmotion-'])
    window['-OUTPUTPATHClassification-'].update('Your Path is for Classify Favourites is :' + values['-PATH_Classify-'])
    #window['-OUTPUTPATHClearFolders-'].update('Your albums are cleared' + values['-PATH_Clear-'])
    
# Finish up by removing from the screen
window.close()

In [3]:
#!pip install mtcnn


# import the necessary packages
from pyimagesearch.blur_detector import detect_blur_fft
import numpy as np
import argparse
import imutils
import cv2
import mtcnn
import matplotlib.pyplot as plt
from imutils import paths
import argparse
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import PySimpleGUI as sg
import os

fileList = glob.glob(r'2_Emotions\*\*.png')
for filePath in fileList:
    try:
        os.remove(filePath)
    except:
        print("Error while deleting file : ", filePath)


folder_path = (r'2b_Not_Face')
test = os.listdir(folder_path)
for images in test:
    if images.endswith(".png"):
        os.remove(os.path.join(folder_path, images))


folder_path = (r'2a_Face')
test = os.listdir(folder_path)
for images in test:
    if images.endswith(".png"):
        os.remove(os.path.join(folder_path, images))


folder_path = (r'1a_Blur_Images')
test = os.listdir(folder_path)
for images in test:
    if images.endswith(".png"):
        os.remove(os.path.join(folder_path, images))


folder_path = (r'1b_No_Blur')
test = os.listdir(folder_path)
for images in test:
    if images.endswith(".png"):
        os.remove(os.path.join(folder_path, images))


fileList1 = glob.glob(r'3a_Classifications\*\*.png')
for filePath in fileList1:
    try:
        os.remove(filePath)
    except:
        print("Error while deleting file : ", filePath)